## OTHER

In [3]:
import meshio
import os
import numpy as np
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.base.base import Struct

dim = 2
float_precision = 'float32'
PATH = os.getcwd()
PATH = os.path.join(PATH, 'mesh', 'triangles.msh')
#PATH = os.path.join(PATH, 'mesh', 'cuboid.msh')

running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
MeshPartition(PATH, dim=dim, conf=running_conf, periodic=[0, 0, 0])
domain = Domain(dim=dim, conf=running_conf)
m_mesh = meshio.read(PATH)

Reading gmsh file ...
Saving partition files ...
Number of Cells: 200
Number of Vertices: 121
Local domain contruction ...


In [4]:
d_cells = domain._cells._nodeid
d_points = domain._nodes._vertex
d_center = domain._cells._center
d_volume = domain._cells._volume

In [ ]:
d_cells.shape

In [ ]:
a = np.array([1, 2, 3, 1])
a[[1, 2, 3]] = 0
a

In [ ]:
d_cells

In [ ]:
cube = np.array([
  [0, 0, 0],
  [1, 1, 1],
  [2, 2, 2],
  [3, 3, 3],
  [4, 4, 4],
  [5, 5, 5],
  [6, 6, 6],
  [7, 7, 7],
], dtype=np.float64)
points = np.array([
  [cube[6], cube[7], cube[2], cube[5]],
  [cube[7], cube[2], cube[5], cube[4]],
  [cube[5], cube[4], cube[2], cube[1]],
  [cube[7], cube[2], cube[4], cube[3]],
  [cube[2], cube[1], cube[4], cube[3]],
  [cube[4], cube[1], cube[0], cube[3]],
])

points


In [ ]:
def main_test(domain, decimal_precision):
  """
  :param domain: manapy domain
  :param decimal_precision: decimal precision for comparison

  :details Create a cuboid mesh constructed from cuboid cells, retrieve the cell center and area, and compare them with the corresponding domain values.
  """
  With = 10.0
  Height = 5.0
  Depth = 15.0
  StepX = With / 10.0
  StepY = Height / 10.0
  StepZ = Depth / 10.0

  d_cells = domain._cells._nodeid
  d_points = domain._nodes._vertex[:, 0:3]
  d_center = domain._cells._center[:, 0:3]
  d_volume = domain._cells._volume

  cmp = 0
  for x in np.arange(With, 0.0, -StepX):
    for z in np.arange(Depth, 0.0, -StepZ):
      for y in np.arange(Height, 0.0, -StepY):
        cuboid_points = np.array([
          [x, y, z],
          [x - StepX, y, z],
          [x - StepX, y, z - StepZ],
          [x, y, z - StepZ],

          [x, y - StepY, z],
          [x - StepX, y - StepY, z],
          [x - StepX, y - StepY, z - StepZ],
          [x, y - StepY, z - StepZ],
        ], dtype=d_points.dtype)

        tetrahedrons = np.array([
          [cuboid_points[0], cuboid_points[1], cuboid_points[3], cuboid_points[4]],
          [cuboid_points[1], cuboid_points[3], cuboid_points[4], cuboid_points[5]],
          [cuboid_points[4], cuboid_points[5], cuboid_points[3], cuboid_points[7]],
          [cuboid_points[1], cuboid_points[3], cuboid_points[5], cuboid_points[2]],
          [cuboid_points[3], cuboid_points[7], cuboid_points[5], cuboid_points[2]],
          [cuboid_points[5], cuboid_points[7], cuboid_points[6], cuboid_points[2]],
        ])
        if cmp == 1:
          print(tetrahedrons)
        cmp += 1

main_test(domain, 4)

In [ ]:
m_mesh.cells['tetra'][0:6]

In [ ]:
m_mesh.cells['hexahedron'][0]

In [ ]:
ids = [6,  62, 197, 106,  61, 520, 602, 431]
id_to_idx = {item: idx for idx, item in enumerate(ids)}
id_to_idx

In [ ]:
np.vectorize(lambda x : id_to_idx.get(x))(m_mesh.cells['tetra'][0:6])

In [ ]:
m_mesh.points[m_mesh.cells['tetra'][11]]

In [ ]:
d_cells[0][:-1]

In [ ]:
c_points = d_points[d_cells[0][:-1]][:, 0:3]
print(c_points)

In [ ]:
def b():
  With = 10.0
  Height = 5.0
  Depth = 15.0
  StepX = With / 10.0
  StepY = Height / 10.0
  StepZ = Depth / 10.0

  cmp = 0
  for x in np.arange(With, 0.0, -StepX):
    for z in np.arange(Depth, 0.0, -StepZ):
      for y in np.arange(Height, 0.0, -StepY):
        points = np.array([
          [x, y, z],
          [x - StepX, y, z],
          [x - StepX, y, z - StepZ],
          [x, y, z - StepZ],

          [x, y - StepY, z],
          [x - StepX, y - StepY, z],
          [x - StepX, y - StepY, z - StepZ],
          [x, y - StepY, z - StepZ],
        ], dtype=d_points.dtype)
        m_points= m_mesh.points[m_mesh.cells['hexahedron'][cmp]]
        np.testing.assert_almost_equal(points, m_points)
        cmp = cmp + 1

b()

In [ ]:
d_cells[[0, 1]][:, :-1]

In [ ]:
m_mesh.points[m_mesh.cells['hexahedron'][[998, 999]]]

In [ ]:
d_cells = domain._cells._nodeid
d_points = domain._nodes._vertex[:, 0:3]
d_center = domain._cells._center[:, 0:3]
d_volume = domain._cells._volume

In [ ]:
d_volume

In [ ]:
d_cells = domain._cells._nodeid
d_points = domain._nodes._vertex[:, 0:3]
d_center = domain._cells._center[:, 0:3]
d_volume = domain._cells._volume

In [ ]:
d_cells_titra = d_cells[d_cells[:, 8] == 4]
d_cells_hexa = d_cells[d_cells[:, 8] == 8]
d_cells_pyra = d_cells[d_cells[:, 8] == 5]

In [ ]:
d_cells[d_cells[:, 8] == 4][:, 0:4]

In [ ]:
d_points[d_cells[0][0:4]]

In [ ]:
def tetrahedron_volume(points : 'float[:, :]'):
  a, b, c, d = points
  matrix = np.array([b - a, c - a, d - a])
  volume = np.abs(np.linalg.det(matrix)) / 6
  return volume

def pyramid_volume(points : 'float[:, :]'):
  tetrahedrons = np.array([
    [
      points[0],
      points[1],
      points[2],
      points[4],
    ],
    [
      points[0],
      points[2],
      points[3],
      points[4],
    ]
  ], dtype=points.dtype)
  return tetrahedron_volume(tetrahedrons[0]) + tetrahedron_volume(tetrahedrons[1])

def hex_volume(points : 'float[:, :]'):
  tetrahedrons = np.array([
    [points[0], points[1], points[3], points[4]],
    [points[1], points[3], points[4], points[5]],
    [points[4], points[5], points[3], points[7]],
    [points[1], points[3], points[5], points[2]],
    [points[3], points[7], points[5], points[2]],
    [points[5], points[7], points[6], points[2]],
  ])
  vol = 0.0
  for tetra_points in tetrahedrons:
    vol += tetrahedron_volume(tetra_points)
  return vol

def get_volume():
  d_cells_titra = d_cells[d_cells[:, 8] == 4][:, 0:4]
  d_cells_hexa = d_cells[d_cells[:, 8] == 8][:, 0:8]
  d_cells_pyra = d_cells[d_cells[:, 8] == 5][:, 0:5]
  volume = 0.0
  for cell in d_cells_titra:
    points = d_points[cell]
    volume += tetrahedron_volume(points)
  for cell in d_cells_hexa:
    points = d_points[cell]
    volume += hex_volume(points)
  for cell in d_cells_pyra:
    points = d_points[cell]
    volume += pyramid_volume(points)
  return volume


In [ ]:
points = np.array([
  [0.0, 0.0, 0.0],
  [1.0, 0.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 0.0, 1.0]
])

tetrahedron_volume(points)

In [ ]:
points = np.array([
  [0.0, 0.0, 0.0],
  [1.0, 0.0, 0.0],
  [1.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.5, 0.5, 3.0],
])

pyramid_volume(points)

In [ ]:
points = np.array([
  [0.0, 0.0, 0.0],
  [1.0, 0.0, 0.0],
  [1.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 0.0, 1.0],
  [1.0, 0.0, 1.0],
  [1.0, 1.0, 1.0],
  [0.0, 1.0, 1.0]
])

hex_volume(points)

In [ ]:
get_volume()

In [ ]:
# Halo

import meshio
import os
import numpy as np
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.base.base import Struct

dim = 2
float_precision = 'float32'
PATH = "/home/aben-ham/Desktop/work/manapy0.4/tests/domain/primary/2D/"
PATH = os.path.join(PATH, 'mesh', 'rectangles.msh')
#PATH = os.path.join(PATH, 'mesh', 'cuboid.msh')

running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
#MeshPartition(PATH, dim=dim, conf=running_conf, periodic=[0, 0, 0])
domain = Domain(dim=dim, conf=running_conf)
m_mesh = meshio.read(PATH)

d_cells = domain._cells._nodeid
d_points = domain._nodes._vertex
d_center = domain._cells._center
d_volume = domain._cells._volume

d_halonid = domain._cells._halonid

print(len(d_cells))
# for i in range(len(d_cells)):
#   if len(d_halonid[i]) != 0:
#     print(d_halonid[i])



In [ ]:
def get_neighboring_by_face(id, width):
  arr = np.array([id - 1, id + 1, id + width, id - width], dtype=np.int32)
  if id % width - 1 < 0:
    arr[0] = -1
  if id % width + 1 >= width:
    arr[1] = -1
  if id + width >= width * width:
    arr[2] = -1
  if id - width < 0:
    arr[3] = -1
  return np.sort(arr[arr != -1])

def get_neighboring_by_vertex(id, width):
  arr = np.array([
    id - 1,
    id + 1,
    id + width,
    id - width,
    id + width - 1,
    id - width - 1,
    id + width + 1,
    id - width + 1,
  ], dtype=np.int32)
  if id % width - 1 < 0:
    arr[[0, 4, 5]] = -1
  if id % width + 1 >= width:
    arr[[1, 6, 7]] = -1
  if id + width >= width * width:
    arr[[2, 4, 6]] = -1
  if id - width < 0:
    arr[[3, 5, 7]] = -1
  return np.sort(arr[arr != -1])

float_precision = 'float32'
running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
domain = Domain(dim=2, conf=running_conf)

size = 5

d_cells = []
d_points = []
d_center = []
d_volume = []
d_cellfid = []
d_cellnid = []
d_halonid = []
d_loctoglob = []
d_node_loctoglob = []
d_halo_halosext = []

for i in range(size):
  domain._create_domain(size, i)
  d_cells.append(domain._cells._nodeid)
  d_points.append(domain._nodes._vertex)
  d_center.append(domain._cells._center)
  d_volume.append(domain._cells._volume)
  d_halonid.append(domain._cells._halonid)
  d_loctoglob.append(domain._cells._loctoglob)
  d_cellfid.append(domain._cells._cellfid)
  d_cellnid.append(domain._cells._cellnid)
  d_node_loctoglob.append(domain._nodes._loctoglob)
  d_halo_halosext.append(domain._halos._halosext)



In [ ]:
domain._create_domain(1, 0)
od_cells = domain._cells._nodeid
od_points = domain._nodes._vertex
od_center = domain._cells._center
od_volume = domain._cells._volume
od_cellfid = domain._cells._cellfid
od_cellnid = domain._cells.cellnid
od_halonid = domain._cells.halonid
od_loctoglob = domain.cells._loctoglob
od_node_loctoglob = domain.nodes._loctoglob

In [ ]:
local_domain_index = 0
halo_cell = d_halonid[local_domain_index][15]
print(d_loctoglob[local_domain_index])
print(halo_cell)
print(d_halo_halosext[local_domain_index][halo_cell[0:halo_cell[-1]]][:, 0])

In [30]:
# #########################
# Halonid test
# #########################


which_partition = np.ndarray(shape=(100), dtype=np.int32)
nb_partitions = 5
width = 10

for p in range(nb_partitions):
  loctoglob = d_loctoglob[p]
  for j in range(len(loctoglob)):
    origin_index = loctoglob[j]
    which_partition[origin_index] = p

for p in range(nb_partitions):
  loctoglob = d_loctoglob[p]
  for cellid in range(len(loctoglob)):
    g_cellid = loctoglob[cellid]
    g_cellnid = get_neighboring_by_vertex(g_cellid, width)
    g_halonid = g_cellnid[which_partition[g_cellnid] != which_partition[g_cellid]]
    g_halonid = np.sort(g_halonid)

    halonid = d_halonid[p][cellid]
    halonid = d_halo_halosext[p][halonid[0:halonid[-1]]][:, 0]
    halonid = np.sort(halonid)

    np.testing.assert_array_equal(g_halonid, halonid)





0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [28]:
g_cellid = d_loctoglob[0][19]
g_cellnid = get_neighboring_by_vertex(g_cellid, width)
g_halonid = g_cellnid[which_partition[g_cellnid] != which_partition[g_cellid]]
g_halonid = np.sort(g_halonid)
g_halonid

array([48, 49], dtype=int32)

In [27]:
which_partition[g_cellnid]

array([0, 0, 0, 2, 2], dtype=int32)

In [ ]:
d_loctoglob[1]

In [ ]:
d_halo_halosext[d_halonid[1][24][0]][0]

In [ ]:
d_loctoglob[1]

In [ ]:
get_neighboring_by_vertex(0, 10)

In [ ]:
for i in range(len(original_domain)):
  loc_index = which_loctoglob[i]
  j = i % 25
  cellnid = d_cellnid[loc_index][j][0:d_cellnid[loc_index][j][-1]]
  halonid = d_halonid[loc_index][j][0:d_halonid[loc_index][j][-1]]
  origin_halonid = d_loctoglob[loc_index][halonid]
  origin_cellnid = d_loctoglob[loc_index][cellnid]
  origin_cellnid = np.concatenate((origin_cellnid, origin_halonid))
  origin_cellnid = np.sort(origin_cellnid)
  true_cellnid = get_neighboring_by_vertex(i, 10)
  np.testing.assert_equal(true_cellnid, origin_cellnid)
  print(i, " cellnid ", origin_cellnid)
  print("true_cellnid", true_cellnid)



In [ ]:
len(which_loctoglob)

## TEST neighboring by face triangle

In [2]:
import meshio
import os
import numpy as np
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.base.base import Struct

dim = 2
float_precision = 'float32'
PATH = os.getcwd()
PATH = os.path.join(PATH, 'mesh', 'triangles.msh')
#PATH = os.path.join(PATH, 'mesh', 'cuboid.msh')

running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
MeshPartition(PATH, dim=dim, conf=running_conf, periodic=[0, 0, 0])
domain = Domain(dim=dim, conf=running_conf)

d_cellnid = domain._cells._cellnid


Reading gmsh file ...
Saving partition files ...
Number of Cells: 200
Number of Vertices: 121
Local domain contruction ...


In [3]:
id = 2
print(np.sort(d_cellnid[id][0:d_cellnid[id][-1]]))
print(get_neighboring_by_vertex(id, 10))

[ 0  1  3  4 20 21 22]
[ 0  1  3  4 20 21 22]


In [7]:
def get_neighboring_by_face(i, width):
  sq_id = i // 2
  if i % 2 == 0:
    # all neighboring cells
    arr = np.array([i + 1, i - 1, i - width * 2 + 1])
    # disable non-existing neighboring cells
    if sq_id % width - 1 < 0:
      arr[1] = -1
    if sq_id - width < 0:
      arr[2] = -1
  else:
    # all neighboring cells
    arr = np.array([i - 1, i + 1, i + width * 2 - 1])
    # disable non-existing neighboring cells
    if sq_id % width + 1 >= width:
      arr[1] = -1
    if sq_id + width >= width * width:
      arr[2] = -1
  return np.sort(arr[arr != -1])

def get_neighboring_by_vertex(i, width):
  sq_id = i // 2

  arr = np.array([
    i - width * 2 - 3,
    i - width * 2 - 2,
    i - width * 2 - 1,
    i - width * 2,
    i - width * 2 + 1,
    i - width * 2 + 2,
    i - 3, #6
    i - 2,
    i - 1, # index 8, neighbor in the same cell
    i + 1,
    i + 2,
    i + width * 2 - 3, #11
    i + width * 2 - 2,
    i + width * 2 - 1,
    i + width * 2,
    i + width * 2 + 1,
    i + width * 2 + 2,
  ], dtype=np.int32)
  if i % 2 == 0:
    arr = arr + 1
    arr[8] = i + 1
    arr[[0, 10, 14, 15, 16]] = -1
  else:
    arr[[0, 1, 2, 6, 16]] = -1
  if sq_id % width - 1 < 0:
    arr[[0, 1, 6, 7, 11, 12]] = -1
  if sq_id % width + 1 >= width:
    arr[[4, 5, 9, 10, 15, 16]] = -1
  if sq_id + width >= width * width:
    arr[[11, 12, 13, 14, 15, 16]] = -1
  if sq_id - width < 0:
    arr[[0, 1, 2, 3, 4, 5]] = -1
  return np.sort(arr[arr != -1])

def main_test(domain, decimal_precision):
  """
  :param domain: manapy domain
  :param decimal_precision: decimal precision for comparison

  :details Create a rectangular mesh using loops, retrieve the cell center and area (each rectangle has two triangles), and compare them with the corresponding domain values.
  """
  Steps = 10
  With = 10.0
  Height = 5.0
  StepX = With / Steps
  StepY = Height / Steps

  d_cells = domain._cells._nodeid
  d_points = domain._nodes._vertex[:, 0:2]
  d_center = domain._cells._center[:, 0:2]
  d_area = domain._cells._volume
  d_cellfid = domain._cells._cellfid
  d_cellnid = domain._cells._cellnid

  cmp = 0
  for x in np.arange(0, With, StepX):
    for y in np.arange(0, Height, StepY):
      p1 = np.array([x, y])
      p2 = np.array([x + StepX, y])
      p3 = np.array([x + StepX, y + StepY])
      p4 = np.array([x, y + StepY])
      area = (StepX * StepY)

      # traingle 1
      t1_points = np.array([p1, p2, p4], dtype=d_points.dtype)
      t1_center = np.array([
        np.sum(t1_points[:, 0]) / 3.0,
        np.sum(t1_points[:, 1]) / 3.0,
      ], dtype=d_center.dtype)
      t1_area = area / 2.0
      t1_nf = get_neighboring_by_face(cmp, Steps)
      t1_nv = get_neighboring_by_vertex(cmp, Steps)

      # traingle 2
      t2_points = np.array([p4, p2, p3])
      t2_center = np.array([
        np.sum(t2_points[:, 0]) / 3.0,
        np.sum(t2_points[:, 1]) / 3.0,
      ], dtype=d_points.dtype)
      t2_area = area / 2.0
      t2_nf = get_neighboring_by_face(cmp + 1, Steps)
      t2_nv = get_neighboring_by_vertex(cmp + 1, Steps)

      # domain traingle 1
      c1_points = d_points[d_cells[cmp][:-1]]
      c1_center = d_center[cmp]
      c1_area = d_area[cmp]
      c1_nf = np.sort(d_cellfid[cmp][0:d_cellfid[cmp][-1]])
      c1_nv = np.sort(d_cellnid[cmp][0:d_cellnid[cmp][-1]])

      # domain traingle 2
      c2_points = d_points[d_cells[cmp + 1][:-1]]
      c2_center = d_center[cmp + 1]
      c2_area = d_area[cmp + 1]
      c2_nf = np.sort(d_cellfid[cmp + 1][0:d_cellfid[cmp + 1][-1]])
      c2_nv = np.sort(d_cellnid[cmp + 1][0:d_cellnid[cmp + 1][-1]])

      np.testing.assert_almost_equal(c1_points, t1_points, decimal=decimal_precision)
      np.testing.assert_almost_equal(c2_points, t2_points, decimal=decimal_precision)
      np.testing.assert_almost_equal(c1_center, t1_center, decimal=decimal_precision)
      np.testing.assert_almost_equal(c2_center, t2_center, decimal=decimal_precision)
      np.testing.assert_almost_equal(c1_area, t1_area, decimal=decimal_precision)
      np.testing.assert_almost_equal(c2_area, t2_area, decimal=decimal_precision)
      np.testing.assert_equal(c1_nf, t1_nf)
      np.testing.assert_equal(c2_nf, t2_nf)
      np.testing.assert_equal(c1_nv, t1_nv)
      np.testing.assert_equal(c2_nv, t2_nv)

      cmp += 2

main_test(domain, decimal_precision=4)

## TEST halos triangle

In [1]:
import meshio
import os
import numpy as np
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.base.base import Struct

dim = 2
float_precision = 'float32'
PATH = os.getcwd()
PATH = os.path.join(PATH, 'mesh', 'triangles.msh')
#PATH = os.path.join(PATH, 'mesh', 'cuboid.msh')

running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
MeshPartition(PATH, dim=dim, conf=running_conf, periodic=[0, 0, 0])
domain = Domain(dim=dim, conf=running_conf)

Reading gmsh file ...
Saving partition files ...
Number of Cells: 200
Number of Vertices: 121
Local domain contruction ...


In [8]:
def get_neighboring_by_face(i, width):
  sq_id = i // 2
  if i % 2 == 0:
    # all neighboring cells
    arr = np.array([i + 1, i - 1, i - width * 2 + 1], dtype=np.int32)
    # disable non-existing neighboring cells
    if sq_id % width - 1 < 0:
      arr[1] = -1
    if sq_id - width < 0:
      arr[2] = -1
  else:
    # all neighboring cells
    arr = np.array([i - 1, i + 1, i + width * 2 - 1], dtype=np.int32)
    # disable non-existing neighboring cells
    if sq_id % width + 1 >= width:
      arr[1] = -1
    if sq_id + width >= width * width:
      arr[2] = -1
  return np.sort(arr[arr != -1])

def get_neighboring_by_vertex(i, width):
  sq_id = i // 2

  arr = np.array([
    i - width * 2 - 3,
    i - width * 2 - 2,
    i - width * 2 - 1,
    i - width * 2,
    i - width * 2 + 1,
    i - width * 2 + 2,
    i - 3, #6
    i - 2,
    i - 1, # index 8, neighbor in the same cell
    i + 1,
    i + 2,
    i + width * 2 - 3, #11
    i + width * 2 - 2,
    i + width * 2 - 1,
    i + width * 2,
    i + width * 2 + 1,
    i + width * 2 + 2,
  ], dtype=np.int32)
  if i % 2 == 0:
    arr = arr + 1
    arr[8] = i + 1
    arr[[0, 10, 14, 15, 16]] = -1
  else:
    arr[[0, 1, 2, 6, 16]] = -1
  if sq_id % width - 1 < 0:
    arr[[0, 1, 6, 7, 11, 12]] = -1
  if sq_id % width + 1 >= width:
    arr[[4, 5, 9, 10, 15, 16]] = -1
  if sq_id + width >= width * width:
    arr[[11, 12, 13, 14, 15, 16]] = -1
  if sq_id - width < 0:
    arr[[0, 1, 2, 3, 4, 5]] = -1
  return np.sort(arr[arr != -1])

# should be created before using mpi
nb_partitions = 5

d_domain = []
d_cells = []
d_cell_faces = []
d_points = []
d_center = []
d_volume = []
d_cellfid = []
d_cellnid = []
d_halonid = []
d_loctoglob = []
d_node_loctoglob = []
d_halo_halosext = []
d_face_halofid = []
d_faces = []
d_face_name = []

for i in range(nb_partitions):
  domain.create_domain(nb_partitions, i)
  d_cells.append(domain.cells.nodeid)
  d_cell_faces.append(domain.cells.faceid)
  d_points.append(domain.nodes.vertex)
  d_center.append(domain.cells.center)
  d_volume.append(domain.cells.volume)
  d_halonid.append(domain.cells.halonid)
  d_loctoglob.append(domain.cells.loctoglob)
  d_cellfid.append(domain.cells.cellfid)
  d_cellnid.append(domain.cells.cellnid)
  d_node_loctoglob.append(domain.nodes.loctoglob)
  d_halo_halosext.append(domain.halos.halosext)
  d_face_halofid.append(domain.faces.halofid)
  d_faces.append(domain.faces.nodeid)
  d_face_name.append(domain.faces.name)



Local domain contruction ...
Local domain contruction ...


In [122]:
local_domain_index = 3
(d_face_halofid[local_domain_index])[d_face_name[local_domain_index] != 10] = -1
target_cell = d_cell_faces[local_domain_index][36]
halo_cell = d_face_halofid[local_domain_index][target_cell[0:3]]
halo_cell = halo_cell[halo_cell != -1]
print(target_cell)
print(d_loctoglob[local_domain_index])
print(halo_cell)
print(d_halo_halosext[local_domain_index][halo_cell][:, 0])

[61 62 52  3]
[ 0  1  2  3  4  5  6  7  8  9 10 20 21 22 23 24 25 26 27 28 29 40 41 42
 43 44 45 46 47 48 60 61 62 63 64 65 66 80 81 82]
[6]
[67]


In [125]:
# #########################
# Halonid test
# #########################


which_partition = np.ndarray(shape=(200), dtype=np.int32)
width = 10

# create which_partition (cell partition id)
for p in range(nb_partitions):
  loctoglob = d_loctoglob[p]
  for j in range(len(loctoglob)):
    origin_index = loctoglob[j]
    which_partition[origin_index] = p

for p in range(nb_partitions):
  loctoglob = d_loctoglob[p]

  # remove any face that does not have a halo cell.
  (d_face_halofid[p])[d_face_name[p] != 10] = -1
  for cellid in range(len(loctoglob)):
    g_cellid = loctoglob[cellid]

    g_cellnid = get_neighboring_by_vertex(g_cellid, width)
    # every neighboring cell not in the same partition is a halo cell
    g_halonid = g_cellnid[which_partition[g_cellnid] != which_partition[g_cellid]]
    g_halonid = np.sort(g_halonid)

    g_cellfid = get_neighboring_by_face(g_cellid, width)
    # every neighboring cell not in the same partition is a halo cell
    g_halofid = g_cellfid[which_partition[g_cellfid] != which_partition[g_cellid]]
    g_halofid = np.sort(g_halofid)

    halonid = d_halonid[p][cellid]
    # get domain global halo cells
    halonid = d_halo_halosext[p][halonid[0:halonid[-1]]][:, 0]
    halonid = np.sort(halonid)

    cell_faces = d_cell_faces[p][cellid]
    halofid = d_face_halofid[p][cell_faces[0:cell_faces[-1]]]
    halofid = halofid[halofid != -1]
    halofid = d_halo_halosext[p][halofid][:, 0]
    halofid = np.sort(halofid)

    #test
    np.testing.assert_array_equal(g_halonid, halonid)
    np.testing.assert_array_equal(g_halofid, halofid)
